#ADE-Net Baseline Training

Code and Paper by: Nicholas Soucy and Dr. Salimeh Yasaei Sekeh

##Initial Setup

### Imports

In [ ]:
import torch

if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import random
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import cohen_kappa_score as kappa
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import utils
from sklearn import mixture
from sklearn.neighbors import KNeighborsClassifier
import sklearn.preprocessing as sp
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM
import os
import os.path as osp
import torch
import torch.utils.data as data
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
from torch import optim
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from operator import truediv
import tensorflow as tf
import math
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
from tqdm import trange
import torch.utils.data as data_utils
from scipy.sparse.linalg import LinearOperator as ScipyLinearOperator
from scipy.sparse.linalg import eigsh
from warnings import warn
import shutil
import sys

from numpy import savetxt
from numpy import loadtxt

Tesla T4
Mounted at /content/drive


### Variables

In [3]:
# GLOBAL VARIABLES TO CHANGE

# Batch Number
batch = 512

# Attack Stuff
attack_types = ['FGSM', 'CW', 'PGD', 'I-FGSM', 'VANILA']
num_attacks = 2

# Number of Epochs
num_epochs = 100

# Number of classes
class_num = 15
cls = class_num

dataset = "KSC"

if dataset == "KSC":
  PATH = "Data/Kennedy_Space_Center/"
elif dataset == "IP":
  PATH = "Data/Indian_Pines/"
elif dataset == "Houston":
  PATH = "Data/Houston/"

print("PATH: ",PATH)

PATH:  /Data/Kennedy_Space_Center/


## Classification Functions

### Unet

#### Pytorch Unet

In [ ]:
class UNet(nn.Module):
    def __init__(self,in_shape,class_num,want_logits=True):
        super().__init__()
        # Define Variables
        self.input_shape = in_shape
        self.class_num = class_num
        self.want_logits = want_logits
        # Define Layers:
        # NOTE: Change kernel size to be the size of the patch
        self.Conv1 = nn.Conv2d(self.input_shape, 64,kernel_size=(1,1), stride=(1,1), padding='same', bias=False)
        self.bn1 = nn.InstanceNorm1d(64)
        self.Conv2 = nn.Conv2d(64, 128,kernel_size=(1,1), stride=(2,2), bias=False)
        self.bn2 = nn.InstanceNorm1d(128)
        self.Conv3 = nn.Conv2d(128, 256,kernel_size=(1,1), stride=(2,2), bias=False)
        self.bn3 = nn.InstanceNorm1d(256)
        self.deco1 = nn.ConvTranspose2d(256,256,kernel_size=(1,1), stride=(1,1), bias=False)
        self.deco2 = nn.ConvTranspose2d(384,128,kernel_size=(1,1), stride=(1,1), bias=False)
        self.deco3 = nn.ConvTranspose2d(192,class_num,kernel_size=(1,1), stride=(1,1))

        # Define Activation functions:
        self.drop = nn.Dropout(0.2)
        self.Lrelu = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=-1)
    def forward(self, x):
        """
        Layers: 4
        Activation Functions:
        RELU for first two layers
        Sigmoid for third layer
        Log Softmax for last layer
        """
        # import pdb;pdb.set_trace()
        x = self.Conv1(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn1(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        op1 = x

        x = self.Conv2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        op2 = x

        x = self.Conv3(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn3(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        op3 = x

        x = self.deco1(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn3(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        x = torch.cat((x, op2), dim=1)

        x = self.deco2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]/2),2)
        # x = self.bn2(x)
        # x = x.view(int(x.shape[0]),int(x.shape[1]*x.shape[2]),1,1)
        x = self.Lrelu(x)
        x = self.drop(x)
        x = torch.cat((x, op1), dim=1)

        x = self.deco3(x)
        x = torch.reshape(x, (x.shape[0], self.class_num))

        if(self.want_logits):
          return x
        else:
          x = self.softmax(x)
          return x

### Training Function

In [ ]:
def single_train_test(train_loader, test_loader, num_epochs=20, class_num=15, learning_rate=0.001):

    for i, (images, labels) in enumerate(train_loader):
        in_shape = images.shape[1]
        break

    model = UNet(in_shape,class_num)
    model.to(device)
    ce = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)




    train_running_loss = [] * num_epochs

    running_acc = [] * num_epochs
    
    # Set parameters for testing
    test_running_acc = [] * num_epochs


    for epoch in range(num_epochs):

        #Training Stuff
        temp_loss = 0

        correct = 0
        total = 0

        #Testing Stuff
        correct_test = 0
        total_test = 0


        ## training step

        # Set networks to train
        model.train()
        
        for idx, (images, labels) in enumerate(train_loader):

            # print("On batch: ",idx)

            images = images.clone().detach().requires_grad_(True)
            labels = labels.clone().detach().requires_grad_(True)
            labels = labels.type(torch.LongTensor)  # casting to long
            images = images.to(device)
            labels = labels.to(device)

            # Calculate gradient and update weights of discriminator network
            optimizer.zero_grad()
            output = model(images)

            loss = ce(output, labels)
            loss.backward()
            optimizer.step()
            temp_loss += loss.detach().item()
            total += labels.size(0)
            _, predicted = torch.max(output.data, 1)
            correct += (predicted == labels).sum().item()




        ## Testing step

        #Set each network to eval
        model.eval()
        for idx, (images, labels) in enumerate(test_loader):

            #Get data
            images = images.clone().detach().requires_grad_(True)
            labels = labels.clone().detach().requires_grad_(True)
            #Send data to device
            images = images.to(device)
            labels = labels.to(device)

            #output
            output = model(images)
            # a_labels = torch.tensor(np.array(tf.math.argmax(a_labels.cpu().clone().detach().numpy(), axis=1))).to(device)
            _, predicted = torch.max(output.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()





        train_running_loss.append(temp_loss / idx)


        if total == 0:
            running_acc.append(0)
        else: 
            running_acc.append((100 * correct / total))

        if total_test == 0:
            test_running_acc.append(0)
        else:        
            test_running_acc.append(100 * (correct_test/total_test))


		#Step Schedulers
        scheduler.step()
        
        print('Epoch: %d | Train Loss: %.4f | Train Acc: %.4f | Testing Acc: %.4f' \
              % (epoch, train_running_loss[epoch], running_acc[epoch], test_running_acc[epoch]))

    return model, train_running_loss, running_acc, test_running_acc



## Loading Data

### Load adversarial and vanilia data

In [ ]:
# Load vanilia and attacked datasets 

# vanilia
vanilia_train_data = np.load('{}VANILLA/x_train.npy'.format(PATH))
vanilia_train_labels = np.load('{}VANILLA/y_train.npy'.format(PATH))
vanilia_test_data = np.load('{}VANILLA/x_test.npy'.format(PATH))
vanilia_test_labels = np.load('{}VANILLA/y_test.npy'.format(PATH))

"My name is {}, I'm {}".format("John",36)

print("Vanilia Data Shape")
print(vanilia_train_data.shape)
print(vanilia_train_labels.shape)
print(vanilia_test_data.shape)
print(vanilia_test_labels.shape)
print("\n")

# fgsm
fgsm_train_data = np.load('{}FGSM/x_adv_train.npy'.format(PATH))
fgsm_train_labels = np.load('{}FGSM/y_train.npy'.format(PATH))
fgsm_test_data = np.load('{}FGSM/x_adv_test.npy'.format(PATH))
fgsm_test_labels = np.load('{}FGSM/y_test.npy'.format(PATH))

print("FGSM Data Shape")
print(fgsm_train_data.shape)
print(fgsm_train_labels.shape)
print(fgsm_test_data.shape)
print(fgsm_test_labels.shape)
print("\n")


#C_W

cw_train_data = np.load('{}CW/x_adv_train.npy'.format(PATH))
cw_train_labels = np.load('{}CW/y_train.npy'.format(PATH))
cw_test_data = np.load('{}CW/x_adv_test.npy'.format(PATH))
cw_test_labels = np.load('{}CW/y_test.npy'.format(PATH))

print("CW Data Shape")
print(cw_train_data.shape)
print(cw_train_labels.shape)
print(cw_test_data.shape)
print(cw_test_labels.shape)
print("\n")

#pgd

pgd_train_data = np.load('{}PGD/x_adv_train.npy'.format(PATH))
pgd_train_labels = np.load('{}PGD/y_train.npy'.format(PATH))
pgd_test_data = np.load('{}PGD/x_adv_test.npy'.format(PATH))
pgd_test_labels = np.load('{}PGD/y_test.npy'.format(PATH))

print("PGD Data Shape")
print(pgd_train_data.shape)
print(pgd_train_labels.shape)
print(pgd_test_data.shape)
print(pgd_test_labels.shape)
print("\n")


#ifgsm

ifgsm_train_data = np.load('{}I-FGSM/x_adv_train.npy'.format(PATH))
ifgsm_train_labels = np.load('{}I-FGSM/y_train.npy'.format(PATH))
ifgsm_test_data = np.load('{}I-FGSM/x_adv_test.npy'.format(PATH))
ifgsm_test_labels = np.load('{}I-FGSM/y_test.npy'.format(PATH))

print("I-FGSM Data Shape")
print(ifgsm_train_data.shape)
print(ifgsm_train_labels.shape)
print(ifgsm_test_data.shape)
print(ifgsm_test_labels.shape)
print("\n")



names = []

# Create master arrays
x_train = [None] * num_attacks
x_test = [None] * num_attacks
y_train = [None] * num_attacks
y_test = [None] * num_attacks

# Merge data and labels
if num_attacks > 0:
	x_train[0] = fgsm_train_data
	y_train[0] = fgsm_train_labels
	x_test[0] = fgsm_test_data
	y_test[0] = fgsm_test_labels
if num_attacks > 1:
	x_train[1] = cw_train_data
	y_train[1] = cw_train_labels
	x_test[1] = cw_test_data
	y_test[1] = cw_test_labels
if num_attacks > 2:
	x_train[2] = pgd_train_data
	y_train[2] = pgd_train_labels
	x_test[2] = pgd_test_data
	y_test[2] = pgd_test_labels
if num_attacks > 3:
	x_train[3] = ifgsm_train_data
	y_train[3] = ifgsm_train_labels
	x_test[3] = ifgsm_test_data
	y_test[3] = ifgsm_test_labels
if num_attacks > 4:
	x_train[4] = vanilia_train_data
	y_train[4] = vanilia_train_labels
	x_test[4] = vanilia_test_data
	y_test[4] = vanilia_test_labels


# Get names and print
print("Attacks Used This Run")
for i in range(num_attacks):
    names.append(attack_types[i])
    print(names[i])
print("\n")


# Show attacked data shape


print("Combined Data Shape")
print(np.shape(x_train))
print(np.shape(x_test))
print("Combined Class Label Shape")
print(np.shape(y_train))
print(np.shape(y_test))
print("Combined Attack Label Shape")
print(np.shape(a_train))
print(np.shape(a_test))
print("\n")



## Training

### Training Baseline

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


def unison_shuffled_copies3(a, b, c):
    assert len(a) == len(b)
    assert len(a) == len(c)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]


In [ ]:
# get full data and shuffle
if num_attacks == 2:
	gt = np.concatenate((np.array(y_train[0]), np.array(y_train[1])))
	data = np.concatenate((np.array(x_train[0]), np.array(x_train[1])))
elif num_attacks == 3:
	gt = np.concatenate((np.array(y_train[0]), np.array(y_train[1]), np.array(y_train[2])))
	data = np.concatenate((np.array(x_train[0]), np.array(x_train[1]), np.array(x_train[2])))
elif num_attacks == 4:
	gt = np.concatenate((np.array(y_train[0]), np.array(y_train[1]), np.array(y_train[2]), np.array(y_train[3])))
	data = np.concatenate((np.array(x_train[0]), np.array(x_train[1]), np.array(x_train[2]), np.array(x_train[3])))
elif num_attacks == 5:
	gt = np.concatenate((np.array(y_train[0]), np.array(y_train[1]), np.array(y_train[2]), np.array(y_train[3]), np.array(y_train[4])))
	data = np.concatenate((np.array(x_train[0]), np.array(x_train[1]), np.array(x_train[2]), np.array(x_train[3]), np.array(x_train[4])))



data, gt = unison_shuffled_copies(data, gt)

# convert to Tensors
gt = torch.Tensor(gt)
data = torch.Tensor(data)

# print(np.shape(gt))
# print(np.shape(data))

# create datasets
dataset = data_utils.TensorDataset(data, gt)
train_loader = data_utils.DataLoader(dataset, batch_size=batch, shuffle=False, pin_memory=True)



# get full data and shuffle
if num_attacks == 2:
	gt_t = np.concatenate((np.array(y_test[0]), np.array(y_test[1])))
	data_t = np.concatenate((np.array(x_test[0]), np.array(x_test[1])))
elif num_attacks == 3:
	gt_t = np.concatenate((np.array(y_test[0]), np.array(y_test[1]), np.array(y_test[2])))
	data_t = np.concatenate((np.array(x_test[0]), np.array(x_test[1]), np.array(x_test[2])))
elif num_attacks == 4:
	gt_t = np.concatenate((np.array(y_test[0]), np.array(y_test[1]), np.array(y_test[2]), np.array(y_test[3])))
	data_t = np.concatenate((np.array(x_test[0]), np.array(x_test[1]), np.array(x_test[2]), np.array(x_test[3])))
elif num_attacks == 5:
	gt_t = np.concatenate((np.array(y_test[0]), np.array(y_test[1]), np.array(y_test[2]), np.array(y_test[3]), np.array(y_test[4])))
	data_t = np.concatenate((np.array(x_test[0]), np.array(x_test[1]), np.array(x_test[2]), np.array(x_test[3]), np.array(x_test[4])))



data_t, gt_t = unison_shuffled_copies(data_t, gt_t)

# convert to Tensors
gt_t = torch.Tensor(gt_t)
data_t = torch.Tensor(data_t)

# print(np.shape(gt))
# print(np.shape(data))

# create datasets
dataset_t = data_utils.TensorDataset(data_t, gt_t)
test_loader = data_utils.DataLoader(dataset_t, batch_size=batch, shuffle=False, pin_memory=True)


### Network Training

In [ ]:
model, train_running_loss, running_acc, test_running_acc = single_train_test(train_loader, test_loader, num_epochs=num_epochs, class_num=class_num)

## Reporting

In [ ]:
print("Training Accuracy: ",max(running_acc))
print("Testing Accuracy: ",max(test_running_acc))

In [ ]:
plt.plot(running_acc)
plt.title('Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.savefig('output/Train_Acc.png', bbox_inches='tight', dpi=400)
plt.clf()
plt.cla()

In [ ]:
plt.plot(test_running_acc)
plt.title('Testing Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.savefig('output/Test_Acc.png', bbox_inches='tight', dpi=400)
plt.clf()
plt.cla()

In [ ]:
plt.plot(train_running_loss)
plt.title('Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig('output/Loss.png', bbox_inches='tight', dpi=400)
plt.clf()
plt.cla()

In [ ]:
np.save("output/train_running_loss", train_running_loss)
np.save("output/running_acc", running_acc)
np.save("output/test_running_acc", test_running_acc)